In [6]:
!pip install -r requirements.txt

  Using cached https://files.pythonhosted.org/packages/b1/69/fcc29820befae2b96fd0b01225577af653e87cd0914634bb2d372a457bd7/pandas-0.25.1-cp37-cp37m-win_amd64.whl
  Using cached https://files.pythonhosted.org/packages/98/14/9c671580d1d344be5f66d81a898aacb028beb2d8a271a731894eada0f839/gin-config-0.2.1.tar.gz
  Using cached https://files.pythonhosted.org/packages/62/fb/1bdf5bef80bb31369306290c308371a64aebf825029520602570135ea7f1/stochastic-0.4.0-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/3a/b6/89c8c2b653dcdd62d2cdf17acc328183d581ee647bccbab8822cc0fd7981/ccxt-1.18.1120-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/f7/08/25e47a53692c2e0dcd2211a493ddfe9007a5cd92e175d6dffa6169a0b392/tensorflow-1.14.0-cp37-cp37m-win_amd64.whl


  ERROR: Could not find a version that satisfies the requirement ray (from -r ../requirements.txt (line 9)) (from versions: none)
ERROR: No matching distribution found for ray (from -r ../requirements.txt (line 9))


In [7]:
!pip install pandas numpy

  Using cached https://files.pythonhosted.org/packages/b1/69/fcc29820befae2b96fd0b01225577af653e87cd0914634bb2d372a457bd7/pandas-0.25.1-cp37-cp37m-win_amd64.whl


## Example "dumb" trading agent

In [21]:

import os
import sys
import warnings

def warn(*args, **kwargs):
    pass

warnings.warn = warn
warnings.simplefilter(action='ignore', category=FutureWarning)

sys.path.append(os.path.dirname(os.path.abspath('')))

from tensortrade.environments import TradingEnvironment
from tensortrade.exchanges.simulated import FBMExchange
from tensortrade.actions import DiscreteActionStrategy
from tensortrade.rewards import SimpleProfitStrategy

exchange = FBMExchange()
action_strategy = DiscreteActionStrategy()
reward_strategy = SimpleProfitStrategy()

env = TradingEnvironment(exchange=exchange,
                         action_strategy=action_strategy,
                         reward_strategy=reward_strategy)

obs = env.reset()
sell_price = 1e9
stop_price = -1

print('Initial portfolio: ', exchange.portfolio)

for i in range(1000):
    action = 0 if obs['close'] < sell_price else 18
    action = 19 if obs['close'] < stop_price else action
    
    if i == 0 or portfolio['BTC'] == 0:
        action = 16
        sell_price = obs['close'] + (obs['close'] / 50)
        stop_price = obs['close'] - (obs['close'] / 50)
    
    obs, reward, done, info = env.step(action)
    executed_trade = info['executed_trade']
    filled_trade = info['filled_trade']
    portfolio = exchange.portfolio
    
    print('Obs: ', obs)
    print('Reward: ', reward)
    print('Portfolio: ', portfolio)
    print('Trade executed: ', executed_trade.trade_type, executed_trade.price, executed_trade.amount)
    print('Trade filled: ', filled_trade.trade_type, filled_trade.price, filled_trade.amount)

AttributeError: module 'tensorflow_estimator.python.estimator.estimator' has no attribute 'EstimatorV2'

In [22]:
!pip install tensortrade

  Using cached https://files.pythonhosted.org/packages/98/14/9c671580d1d344be5f66d81a898aacb028beb2d8a271a731894eada0f839/gin-config-0.2.1.tar.gz


  ERROR: Could not find a version that satisfies the requirement ray (from tensortrade) (from versions: none)
ERROR: No matching distribution found for ray (from tensortrade)


In [ ]:
import sys
import os
import warnings

def warn(*args, **kwargs):
    pass

warnings.warn = warn
warnings.simplefilter(action='ignore', category=FutureWarning)

import gym
import numpy as np

from tensorforce.agents import Agent
from tensorforce.execution import Runner
from tensorforce.contrib.openai_gym import OpenAIGym

sys.path.append(os.path.dirname(os.path.abspath('')))

from tensortrade.environments import TradingEnvironment
from tensortrade.exchanges.simulated import FBMExchange
from tensortrade.actions import DiscreteActionStrategy
from tensortrade.rewards import SimpleProfitStrategy

exchange = FBMExchange(times_to_generate=100000)
action_strategy = DiscreteActionStrategy()
reward_strategy = SimpleProfitStrategy()

env = TradingEnvironment(exchange=exchange,
                         action_strategy=action_strategy,
                         reward_strategy=reward_strategy,
                         feature_pipeline=None)

agent_config = {
    "type": "dqn_agent",

    "update_mode": {
        "unit": "timesteps",
        "batch_size": 64,
        "frequency": 4
    },
    
    "memory": {
        "type": "replay",
        "capacity": 10000,
        "include_next_states": True
    },

    "optimizer": {
        "type": "clipped_step",
        "clipping_value": 0.1,
        "optimizer": {
            "type": "adam",
            "learning_rate": 1e-3
        }
    },

    "discount": 0.999,
    "entropy_regularization": None,
    "double_q_model": True,

    "target_sync_frequency": 1000,
    "target_update_weight": 1.0,

    "actions_exploration": {
        "type": "epsilon_anneal",
        "initial_epsilon": 0.5,
        "final_epsilon": 0.,
        "timesteps": 1000000000
    },

    "saver": {
        "directory": None,
        "seconds": 600
    },
    "summarizer": {
        "directory": None,
        "labels": ["graph", "total-loss"]
    },
    "execution": {
        "type": "single",
        "session_config": None,
        "distributed_spec": None
    }
}

network_spec = [
    dict(type='dense', size=64),
    dict(type='dense', size=32)
]

agent = Agent.from_spec(
        spec=agent_config,
        kwargs=dict(
            states=env.states,
            actions=env.actions,
            network=network_spec,
        )
    )

# Create the runner
runner = Runner(agent=agent, environment=env)


# Callback function printing episode statistics
def episode_finished(r):
    print("Finished episode {ep} after {ts} timesteps (reward: {reward})".format(ep=r.episode, ts=r.episode_timestep,
                                                                                 reward=r.episode_rewards[-1]))
    return True


# Start learning
runner.run(episodes=300, max_episode_timesteps=10000, episode_finished=episode_finished)
runner.close()

# Print statistics
print("Learning finished. Total episodes: {ep}. Average reward of last 100 episodes: {ar}.".format(
    ep=runner.episode,
    ar=np.mean(runner.episode_rewards))
)

W0827 22:51:22.413044 4475143616 deprecation_wrapper.py:119] From /usr/local/lib/python3.7/site-packages/tensorforce/core/optimizers/tf_optimizer.py:31: The name tf.train.AdadeltaOptimizer is deprecated. Please use tf.compat.v1.train.AdadeltaOptimizer instead.

W0827 22:51:22.413619 4475143616 deprecation_wrapper.py:119] From /usr/local/lib/python3.7/site-packages/tensorforce/core/optimizers/tf_optimizer.py:32: The name tf.train.AdagradOptimizer is deprecated. Please use tf.compat.v1.train.AdagradOptimizer instead.

W0827 22:51:22.414273 4475143616 deprecation_wrapper.py:119] From /usr/local/lib/python3.7/site-packages/tensorforce/core/optimizers/tf_optimizer.py:33: The name tf.train.AdamOptimizer is deprecated. Please use tf.compat.v1.train.AdamOptimizer instead.

W0827 22:51:22.980015 4475143616 lazy_loader.py:50] 
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/201809

Finished episode 1 after 1665 timesteps (reward: -2308.5202051242336)
Finished episode 2 after 1665 timesteps (reward: -1283.0615648964729)
Finished episode 3 after 1665 timesteps (reward: -1776.6452968587246)
Finished episode 4 after 1665 timesteps (reward: -1641.6904180166866)
Finished episode 5 after 1665 timesteps (reward: -1462.7058570205527)
Finished episode 6 after 1665 timesteps (reward: -1518.935115373318)
Finished episode 7 after 1665 timesteps (reward: -1226.6336069671968)
Finished episode 8 after 1665 timesteps (reward: -1403.566511120577)
Finished episode 9 after 1665 timesteps (reward: -1706.5460019310076)
Finished episode 10 after 1665 timesteps (reward: -1563.9667490620047)
Finished episode 11 after 1665 timesteps (reward: -1353.4441234147248)
Finished episode 12 after 1665 timesteps (reward: -1558.8286621450547)
